In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass, field
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from datetime import datetime

from typing import List

import requests
from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import Frame, RGBCameraCapture
from typing import Dict, List, Tuple

import torch
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from torchvision.models import resnet18, ResNet18_Weights

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

np.set_printoptions(suppress=True)
# %matplotlib ipympl

c:\Users\Leonard\miniconda3\envs\openmmlab\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [17]:
class CustomConverter(SOLO2COCOConverter):
    
    def callback(self, results):
        map(super().callback, results)

    @staticmethod
    def _process_instances(
        frame: Frame, idx, output, data_root, solo_kp_map
    ) -> Tuple[Dict, List, List, List]:
        # logger.info(f"Processing Frame number: {idx}")
        image_id = idx
        sequence_num = frame.sequence
        rgb_captures = list(
            filter(lambda cap: isinstance(cap, RGBCameraCapture), frame.captures)
        )

        results = []
        for capture_idx, rgb_capture in enumerate(rgb_captures):
            img_record = SOLO2COCOConverter._process_rgb_image(
                image_id*len(rgb_captures)+capture_idx, rgb_capture, output, data_root, sequence_num
                # f'{image_id}_{capture_idx}', rgb_capture, output, data_root, sequence_num
            )
            (
                ann_record,
                ins_ann_record,
                sem_ann_record,
            ) = SOLO2COCOConverter._process_annotations(
                image_id*len(rgb_captures)+capture_idx, rgb_capture, sequence_num, data_root, solo_kp_map
                # f'{image_id}_{capture_idx}', rgb_capture, sequence_num, data_root, solo_kp_map
            )
            results.append(
                (img_record, ann_record, ins_ann_record, sem_ann_record)
            )
        return results
    
    def convert(self, output_path: str, dataset_name: str = "coco"):
        output = os.path.join(output_path, dataset_name)

        solo_kp_map = self._get_solo_kp_map()

        for idx, frame in enumerate(self._solo.frames()):
            results = self._process_instances(frame, idx, output, self._solo.data_path, solo_kp_map)
            self.callback(results)
            for result in results:
                print(result[0])
            break

        # for idx, frame in enumerate(self._solo.frames()):
        #     self._pool.apply_async(
        #         self._process_instances,
        #         args=(frame, idx, output, self._solo.data_path, solo_kp_map),
        #         callback=self.callback,
        #     )
        self._pool.close()
        self._pool.join()

        self._write_out_annotations(output)

folder_dir = 'C:/Users/Leonard/AppData/LocalLow/DefaultCompany/Perception2/solo_16'

solo = Solo(data_path=folder_dir)
dataset = CustomConverter(solo)
dataset.convert(output_path='./data/')

{'id': 0, 'width': 3000.0, 'height': 3000.0, 'file_name': 'camera_0.png'}
{'id': 1, 'width': 3000.0, 'height': 3000.0, 'file_name': 'camera_1.png'}


In [18]:
dataset._images

[]

In [5]:

folder_dir = 'C:/Users/Leonard/AppData/LocalLow/DefaultCompany/Perception2/solo_16'

solo = Solo(data_path=folder_dir)
output = os.path.join(solo.data_path, 'coco')
# solo_kp_map = self._get_solo_kp_map()

for idx, frame in enumerate(solo.frames()):
    image_id = idx
    sequence_num = frame.sequence
    for capture_idx, rgb_capture in enumerate(frame.captures):
        img_record = SOLO2COCOConverter._process_rgb_image(
            image_id*len(frame.captures)+capture_idx, rgb_capture, output, solo.data_path, sequence_num
        )
        (
            ann_record,
            ins_ann_record,
            sem_ann_record,
        ) = SOLO2COCOConverter._process_annotations(
            image_id*len(frame.captures)+capture_idx, rgb_capture, sequence_num, solo.data_path, solo_kp_map
        )
        # results.append(
        #     (img_record, ann_record, ins_ann_record, sem_ann_record)
        # )
        break
    break

step0.camera_0.png


In [ ]:

def post_process_result(ret):
    for u in range(len(ret[1])):
        for v in range(len(ret[1][u])):
            # remove segmentation result
            ret[1][u][v] = np.zeros_like(ret[1][u][v], dtype=bool)
            # not remove detection result if it is 'Human' class (idx: 0)
            if u == 0: continue
            # remove detection result
            ret[0][u][v] = np.zeros_like(ret[0][u][v], dtype=np.float32)
    return ret

for i, frame in enumerate(solo.frames()):
    for capture in frame.captures:
        # for bbox in capture.annotations:
        #     print(bbox.values)
        label_dict = {
            "images": {
                "id": 1,
                "width": capture.dimension[0],
                "height": capture.dimension[1],
                "file_name": capture.filename
            }
        }
        coco_annotation = COCOeval(annotation_file=coco_annotation_file_path)

        # result = inference_detector(obj_det, f'{folder_dir}/sequence.{i}/{capture.filename}')
        # result = post_process_result(result)

        break
    break

In [ ]:
python tools/test.py "./configs/swin/mask_rcnn_swin-t-p4-w7_fpn_fp16_ms-crop-3x_coco.py" "./checkpoints/mask_rcnn_swin-t-p4-w7_fpn_fp16_ms-crop-3x_coco_20210908_165006-90a4008c.pth" --eval bbox

In [ ]:

"categories": [
    {
        "supercategory": "default",
        "id": 1,
        "name": "person",
        "keypoints": [],
        "skeleton": []
    },
]